In [139]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

train = pd.read_csv("train.csv", index_col=0) # 学習用データ
test = pd.read_csv("test.csv", index_col=0)   # 評価用データ
sample_submit = pd.read_csv("sample_submit.csv", index_col=0, header=None) # 応募用サンプルファイル

In [140]:
# train.info()

In [141]:
# train.corrwith(train["Attrition"])

In [142]:
BusinessTravel={"Non-Travel":1,"Travel_Frequently":2, "Travel_Rarely":3}
train["BusinessTravel"]=train["BusinessTravel"].map(BusinessTravel)
test["BusinessTravel"]=test["BusinessTravel"].map(BusinessTravel)

Department={"Human Resources":1,"Research & Development":2,"Sales":3}
train["Department"]=train["Department"].map(Department)
test["Department"]=test["Department"].map(Department)

EducationField={"Human Resources":1,"Life Sciences":2,"Marketing":3,"Medical":4,"Other":5,"Technical Degree":6}
train["EducationField"]=train["EducationField"].map(EducationField)
test["EducationField"]=test["EducationField"].map(EducationField)


Gender={"Female":1,"Male":2}
train["Gender"]=train["Gender"].map(Gender)
test["Gender"]=test["Gender"].map(Gender)


MaritalStatus={"Divorced":1,"Married":2,"Single":3}
train["MaritalStatus"]=train["MaritalStatus"].map(MaritalStatus)
test["MaritalStatus"]=test["MaritalStatus"].map(MaritalStatus)

Over18={"N":1,"Y":2}
train["Over18"]=train["Over18"].map(Over18)
test["Over18"]=test["Over18"].map(Over18)

OverTime={"No":1,"Yes":2}
train["OverTime"]=train["OverTime"].map(OverTime)
test["OverTime"]=test["OverTime"].map(OverTime)

JobRole={"Research Scientist":1,"Sales Executive":2,"Laboratory Technician":3,"Manufacturing Director":4,"Sales Representative":5,"Healthcare Representative":6,"Research Director":7,"Manager":8,"Human Resources":9}
train["JobRole"]=train["JobRole"].map(JobRole)
test["JobRole"]=test["JobRole"].map(JobRole)


In [143]:
# train.corrwith(train["Attrition"])

In [144]:
from sklearn.preprocessing import StandardScaler

# 標準化の対象となる特徴量を選択
selected_features = ["Age", "DailyRate","DistanceFromHome","EmployeeNumber","HourlyRate","MonthlyIncome","NumCompaniesWorked","PercentSalaryHike","PerformanceRating","RelationshipSatisfaction","StandardHours","StockOptionLevel","TotalWorkingYears","TrainingTimesLastYear","WorkLifeBalance","YearsAtCompany","YearsInCurrentRole","YearsSinceLastPromotion","YearsWithCurrManager"]

# StandardScalerのインスタンスを作成
scaler = StandardScaler()
# 選択した特徴量の平均値と標準偏差を計算し、標準化を実行
train[selected_features] = scaler.fit_transform(train[selected_features])
test[selected_features] = scaler.transform(test[selected_features])


In [145]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train= scaler.fit_transform(train)
test= scaler.transform(test)

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- Attrition


In [127]:
train["JobRole"].value_counts()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [128]:
train.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [90]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.metrics import accuracy_score
test= test.drop(["JobRole"], axis=1)
y = train["Attrition"] # 目的変数
X = train.drop(["Attrition","JobRole"], axis=1) # 目的変数を除いたデータ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# LightGBMデータセットの作成
train_data = lgb.Dataset(X_train, label=y_train)
# パラメータの設定
params = {
    'objective': 'binary',
    'metric': 'binary_logloss'
}

In [91]:
best_ac = 0.0
best_num_boost_round = 0

for num_boost_round in range(1,50):
    # モデルの学習
    model = lgb.train(params, train_data, num_boost_round)
    # テストデータの予測
    y_pred = model.predict(X_test)
    y_pred_binary = [1 if i >= 0.5 else 0 for i in y_pred]
    # 精度の評価
    accuracy = accuracy_score(y_test, y_pred_binary)
    if accuracy > best_ac:
        best_ac = accuracy
        best_num_boost_round = num_boost_round
        
# 最適なnum_boost_roundを表示
print("Best num_boost_round:", best_num_boost_round)
print("Best accuracy:", best_ac)


# model = lgb.train(params, train_data)
# # model = LogisticRegression()
# # model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# y_pred_binary = [1 if i >= 0.5 else 0 for i in y_pred]

# # 精度の評価
# accuracy = accuracy_score(y_test, y_pred_binary)
# print("Accuracy:", accuracy)

# # 評価データの予測
# pred = model.predict(test)
# pred_binary = [1 if i >= 0.5 else 0 for i in pred]
# pred_binary


[LightGBM] [Info] Number of positive: 165, number of negative: 795
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001475 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1024
[LightGBM] [Info] Number of data points in the train set: 960, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.171875 -> initscore=-1.572397
[LightGBM] [Info] Start training from score -1.572397
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 165, number of negative: 795
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000363 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1024
[LightGBM] [Info] Number of data points in the train set: 960, number of used features: 29
[LightGBM] [I

In [92]:
sample_submit[1] = pred_binary
sample_submit.to_csv('submit.csv', header=None, sep=',')